In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv", parse_dates=['datetime'])
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv", parse_dates=['datetime'])
sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")

In [ ]:
new_tr = train.copy()
new_test = test.copy()

In [ ]:
## 더미변수, 파생변수 생성
new_tr['year'] = new_tr['datetime'].dt.year
new_tr['month'] = new_tr['datetime'].dt.month
new_tr['day'] = new_tr['datetime'].dt.day
new_tr['hour'] = new_tr['datetime'].dt.hour
new_tr['minute'] = new_tr['datetime'].dt.minute
new_tr['second'] = new_tr['datetime'].dt.second
new_tr['dayofweek'] = new_tr['datetime'].dt.dayofweek

In [ ]:
new_test['year'] = new_test['datetime'].dt.year
new_test['month'] = new_test['datetime'].dt.month
new_test['day'] = new_test['datetime'].dt.day
new_test['hour'] = new_test['datetime'].dt.hour
new_test['minute'] = new_test['datetime'].dt.minute
new_test['second'] = new_test['datetime'].dt.second
new_test['dayofweek'] = new_test['datetime'].dt.dayofweek

In [ ]:
col_names = ['year','month','day','hour','dayofweek']
i = 0
plt.figure(figsize=(12,35))  ##전체 그래프 크기 지정

for name in col_names: ## 컬럼명으로 반복
  i = i+1
  plt.subplot(6,2,i)  ##2행2열, i = 1,2,3,4 (왼쪽 상단부터 시계방향으로 순번 지정)
  sns.countplot(x = name, data = new_tr)
  plt.title("train feature")
    
  i = i+1
  plt.subplot(6,2,i)  ##2행2열, i = 1,2,3,4 (왼쪽 상단부터 시계방향으로 순번 지정)
  sns.countplot(x = name, data = new_test)
  plt.title("test feature")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
g = sns.heatmap(new_tr.corr(), annot=True, fmt=".2f", cmap="coolwarm", cbar=False)

In [ ]:
feature_names = [ 'season', 'holiday', 'workingday', 'weather', 
                  'temp', 'atemp', 'humidity', 'windspeed', 
                  "year", "hour", "dayofweek"]  # 공통 변수

In [ ]:
### 피처 선택 X, y, test 데이터 셋에 대해 데이터 나누기,
# y - log_count를 생성하고 지정하기

In [ ]:
new_tr['log_count'] = np.log1p(new_tr['count'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# new_tr, new_test
feature_names = [ 'season', 'holiday', 'workingday', 'weather', 
                  'temp', 'atemp', 'humidity', 'windspeed', 
                  "year", "hour", "dayofweek"]  # 공통 변수
X = new_tr[feature_names]
y = new_tr['log_count']

X_test_last = new_test[feature_names]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.1,
                                                   random_state=77)



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import time

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

scores = cross_val_score(model, X_test, y_test, cv=5,
                        scoring='neg_mean_squared_error')
print("MSE 평균 : ",np.abs( scores.mean() ))

In [ ]:
def model_val(model_name, model_obj):
    now_time = time.time()
    
    model_obj.fit(X_train, y_train)
    
    scores = cross_val_score(model_obj, X_test, y_test, cv=5,
                            scoring = 'neg_mean_squared_error')
    avg_score = np.abs( scores.mean() )
    print("MSE 평균 : ", avg_score)
    
    pro_time = time.time() - now_time
    print("수행 시간 : {0:3f}".format(pro_time))
    print("{} Score : {}".format(model_name, avg_score))
    return avg_score, pro_time

In [ ]:
model_list = ["RandomForestRegressor",  "xgb_basic", 
              "lightgbm-model1", "lightgbm-model2"]

exe_model = []
model_score = []
model_time = []

In [ ]:
m_name = model_list[0]

if m_name not in exe_model:
    model = RandomForestRegressor(random_state=30)
    mse_score, p_time = model_val(m_name, model)
    
    exe_model.append(m_name) #실행 완료
    model_score.append(mse_score)
    model_time.append(p_time)
else:
    print(f"{m_name} 이미 실행 완료")


In [ ]:
m_name = model_list[1]

# 점수 및 걸린 시간
if m_name not in exe_model:
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, # 각나무마다 사용하는 feature 비율
                learning_rate = 0.1,
                max_depth = 3, 
                alpha = 0.1, 
                n_estimators = 1000)  # n_estimators=100

    mse_score, p_time = model_val(m_name, xg_reg)
    
    exe_model.append(m_name)        # 실행된 모델
    model_score.append(mse_score)   # 실행된 모델 점수
    model_time.append(p_time)       # 실행된 모델 걸린 시간
else:
    print(f"{m_name} 이미 실행 완료")

In [ ]:
import lightgbm as lgb 

m_name = model_list[2]

# 점수 및 걸린 시간
if m_name not in exe_model:
    m_lgbm1 = lgb.LGBMRegressor()

    mse_score, p_time = model_val(m_name, m_lgbm1)
    
    exe_model.append(m_name)        # 실행된 모델
    model_score.append(mse_score)   # 실행된 모델 점수
    model_time.append(p_time)       # 실행된 모델 걸린 시간
else:
    print(f"{m_name} 이미 실행 완료")

In [ ]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20, 
                   'num_leaves': 56, 
                   'reg_alpha': 0.7543896477745794, 
                   'reg_lambda': 0.07152751159655985, 
                   'subsample_for_bin': 240000, 
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 2000}

m_name = model_list[3]

# 점수 및 걸린 시간
if m_name not in exe_model:
    m_lgbm2 = lgb.LGBMRegressor(**hyperparameters)

    mse_score, p_time = model_val(m_name, m_lgbm2)
    
    exe_model.append(m_name)        # 실행된 모델
    model_score.append(mse_score)   # 실행된 모델 점수
    model_time.append(p_time)       # 실행된 모델 걸린 시간
else:
    print(f"{m_name} 이미 실행 완료")

In [ ]:
import pandas as pd

dict_dat = {"model_name":exe_model, "score":model_score, "time":model_time}
dat = pd.DataFrame(dict_dat)
dat

In [ ]:
sub

In [ ]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20, 
                   'num_leaves': 56, 
                   'reg_alpha': 0.7543896477745794, 
                   'reg_lambda': 0.07152751159655985, 
                   'subsample_for_bin': 240000, 
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 2000}

# 최종 모델 선택 및 제출
m_lgbm2 = lgb.LGBMRegressor(**hyperparameters)
m_lgbm2.fit(X_train, y_train)

pred = m_lgbm2.predict(X_test_last)
sub['count'] = np.expm1(pred)
sub.to_csv("submission.csv", index=False)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(1,2,1)
sns.barplot(x='score', y= 'model_name', data=dat)

plt.subplot(1,2,2)
sns.barplot(x='time', y='model_name', data=dat)